In [64]:
import re
from collections import Counter
import random

In [74]:
"=====CONSTANTS====="
BOOK_FNAME = "AnnaKarenina.txt"
cyrillic_symbols = " абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
cyrillic_symbols_list = list("абвгдеёжзийклмнопрстуфхцчшщъыьэюя")
msg = "меня зовут максимус деций меридий командующий северными армиями генерал легиона феликс верный слуга истинного императора марка аврелия отец убитого сына муж убитой жены и я отомщу за них в этой жизни или следующей"

**1. Реализуйте базовый частотный метод по Шерлоку Холмсу:**

подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);

возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;

расшифруйте их таким частотным методом.

In [75]:
def create_row_from_file():
  with open(BOOK_FNAME, "r") as fin:
    text = fin.readlines()
  one_row = " ".join(text)
  cleaned_row = re.sub(r"\W+", " ", one_row.lower())
  return cleaned_row
prepared_text = create_row_from_file()
letter_freqs = Counter(prepared_text)

In [76]:
def get_accuracy(msg, decoded_msg):
    match = 0
    for x, y in zip(msg, decoded_msg):
        if x == y:
            match += 1
    return match / len(msg)

def encode(msg, row):
    lett_list = list(row)
    random.shuffle(row)
    translation = str.maketrans("".join(lett_list), "".join(row))
    return msg.translate(translation)

def decode(encoded_msg, freqs, key):
    if key == 'word':
      enc_freqs = Counter(encoded_msg)
    elif key == 'bigram':
      enc_freqs = get_bigram_freqs(encoded_msg)
    sort_enc_freqs = [item for item, _ in sorted(enc_freqs.items(), key=lambda letter: (-letter[1], letter[0]))]
    sort_freqs = [item for item, _ in sorted(freqs.items(), key=lambda freq: (-freq[1], freq[0]))]
    translation = str.maketrans("".join(sort_enc_freqs), "".join(sort_freqs[:len(sort_enc_freqs)]))
    return encoded_msg.translate(translation)

def get_bigram_freqs(row):
    bigram_freqs = Counter()
    for idx in range(len(row) - 1):
        bigram_freqs[row[idx:idx + 2]] += 1
    return bigram_freqs

def print_stat(msg, enc, dec):
  print(f"Сообщение:\t{msg}")
  print(f"Закодированная строка:\t{enc}")
  print(f"Раскодированная строка:\t{dec}")
  print(f"Метрика: {get_accuracy(msg, dec)}")


In [77]:
enc_ru_msg = encode(msg, cyrillic_symbols_list)
dec_ru_msg = decode(msg, letter_freqs, 'word')
print_stat(msg, enc_ru_msg, dec_ru_msg)

Сообщение:	меня зовут максимус деций меридий командующий северными армиями генерал легиона феликс верный слуга истинного императора марка аврелия отец убитого сына муж убитой жены и я отомщу за них в этой жизни или следующей
Закодированная строка:	ючнт цусие юолпзюип дчшзя ючьздзя луюондижэзя пчсчьнаюз оьюзтюз фчнчьой йчфзуно щчйзлп счьная пйифо зпезннуфу зюгчьоеуьо юоьло осьчйзт уечш ирзеуфу пано юиъ ирзеуя ъчна з т уеуюэи цо нзы с ёеуя ъзцнз зйз пйчдижэчя
Раскодированная строка:	иеты бнмлр иаявоилв пежок иесопок яниатплйчок веместьио асиоыио уетесад деуонта хедояв местьк вдлуа овроттнун оишесарнса иасяа амседоы нреж лзорнун вьта илг лзорнк геть о ы нрничл ба тою м эрнк гобто одо вдеплйчек
Метрика: 0.29107981220657275


**2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:**

подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

проведите тестирование аналогично п.1, но при помощи биграмм.



In [78]:
bigram_ru_freqs = get_bigram_freqs(prepared_text)
dec_birgam_msg = decode(enc_ru_msg, bigram_ru_freqs, 'bigram')
print_stat(msg, enc_ru_msg, dec_birgam_msg)

Сообщение:	меня зовут максимус деций меридий командующий северными армиями генерал легиона феликс верный слуга истинного императора марка аврелия отец убитого сына муж убитой жены и я отомщу за них в этой жизни или следующей
Закодированная строка:	ючнт цусие юолпзюип дчшзя ючьздзя луюондижэзя пчсчьнаюз оьюзтюз фчнчьой йчфзуно щчйзлп счьная пйифо зпезннуфу зюгчьоеуьо юоьло осьчйзт уечш ирзеуфу пано юиъ ирзеуя ъчна з т уеуюэи цо нзы с ёеуя ъзцнз зйз пйчдижэчя
Раскодированная строка:	аенойэтмряйаиеюуарюйьеиууйаееуьууйетаиньрдмууйюемеенгауйиеауоауй енееилйле утниймелуеюймеенгуйюлр ийуюяуннт тйуаоееиятеийаиееийимеелуойтяеийр уят тйюгнийарсйр уятуйсенгйуйойтятамрйэийнуеймйаятуйсуэнуйулуйюлеьрдмеу
Метрика: 0.1643192488262911


**3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:**

предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;

реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Давайте будем использовать в качестве состояния цепи Маркова символ, тогда можно будет построить матрицу переходов T.
Для строки s вычислим порождения: $P(s) = \prod_i p(s_{i+1}|s_i) = \prod_i T_{s_i,s_{i+1}}$.

Теперь найдем такую перестановку букв, при применении которой к закодированному сообщению получим результат с наивысшим P.

Применение MCMC для поиска этой перестановки: поменяем два символа местами. Затем рассчитаем P2/P1 для исходной и полученной перестановок. Используем метод Метрополиса-Гастингса для принятия решения, оставить ли новую перестановку (P2>P1) или же вернуть старую. Делаем несколько таких итераций и сохраняем результат. Повторяем, пока не получим лучшую перестановку.

In [79]:
def create_matrix(prep_t):
  T = Counter()
  for i in range(len(prep_t) - 1):
      T[(prep_t[i], prep_t[i + 1])] += 1
  for let_1 in cyrillic_symbols:
      for let_2 in cyrillic_symbols:
          T[(let_1, let_2)] += 1
  total = len(prep_t) - 1 + len(cyrillic_symbols) ** 2
  for k in T.keys():
      T[k] /= total
  return T

def get_perms(symbols):
    shuff_symbols = list(symbols)
    random.shuffle(shuff_symbols)
    shuff_symbols = ''.join(shuff_symbols)
    sym_dict = {k: v for k, v in zip(symbols, shuff_symbols)}
    return sym_dict

def get_rpob_ratio(T, text_2, text_1):
    ratio = 1
    for i in range(len(text_2) - 1):
        ratio *= T[(text_2[i], text_2[i + 1])] / T[(text_1[i], text_1[i + 1])]
    return ratio

def decode(enc_msg, T, max_iters=10000, max_attempts=20):
    final = enc_msg
    for j in range(max_attempts):
        perm = get_perms(cyrillic_symbols)
        for i in range(max_iters):
            key_1, key_2= random.choice(list(perm.keys())), random.choice(list(perm.keys()))
            text_1 = ''.join([perm[sym] for sym in enc_msg])
            perm[key_1], perm[key_2] = perm[key_2], perm[key_1]
            text_2 = ''.join([perm[sym] for sym in enc_msg])
            ratio = get_rpob_ratio(T, text_2, text_1)
            if random.uniform(0, 1) > ratio:
                perm[key_1], perm[key_2] = perm[key_2], perm[key_1]
        res = ''.join([perm[sym] for sym in enc_msg])
        if get_rpob_ratio(T, res, final) > 1:
            final = res
    return final

In [82]:
encrypted = encode(msg, cyrillic_symbols_list)
t_matr = create_matrix(prepared_text)
decrypted = decode(encrypted, t_matr)
print(decrypted)
print(f'Accuracy: {get_accuracy(msg, decrypted)}')

веня зомут ваксивус дегий веридий ковандуюший семерныви арвияви пенерал лепиона беликс мерный слупа истиннопо ивьератора варка амрелия отег ужитопо сына вуч ужитой чены и я отовшу за них м этой чизни или следуюшей
Accuracy: 0.8403755868544601


In [83]:

msg = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
transcoder = {k: v for k, v in zip(set(msg), cyrillic_symbols)}
message_ru = ''.join([transcoder[k] for k in msg])

In [84]:
t_matr = create_matrix(prepared_text)
decrypted = decode(message_ru, t_matr)
print(decrypted)

если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
